In [1]:
sentences = [
    "오늘 날씨가 좋아서 나들이 가고 싶다.",
    "이 영화는 정말 재미있었어요.",
    "맛있는 음식을 먹으러 갈까요?",
    "운동을 하면 건강에 좋아지는 것 같아요.",
    "공부하기 싫어서 미루고 있어요.",
    "여행 계획을 세우고 있는데 어디로 갈까요?",
    "좋은 책을 읽으면 마음이 편안해져요.",
    "오늘은 친구들과 만나서 재미있게 놀았어요.",
    "새로운 언어를 배우는 것은 어려워도 흥미로워요.",
    "주말에 가족들과 함께 시간을 보내기로 했습니다."
]

### 자연어 전처리

In [2]:
# %pip install python-mecab-ko

In [3]:
from mecab import MeCab
mecab = MeCab()

In [4]:
# 불용어 리스트 생성 (예시)
stopwords = ['가', '고', '을', '를', '이', '는']
from konlpy.tag import Okt

# Okt 형태소 분석기 인스턴스 생성
okt = Okt()

# 불용어 리스트 생성 (예시)
stopwords = ['가', '고', '을', '를', '이', '는']

# 토크나이징 함수 정의
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword

    ]

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=tokenizer, use_idf=True)
features = vectorizer.fit_transform(sentences)
features.toarray()

# vectorizer를 하면 각 순서(index)와 위치값, 단어들을 가지고 있음
# vectorizer는 거리를 가지고 논다고 생각하자!
# vectorizer를 하고 나면 거리를 잴 수 있음

array([[0.46015789, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.46015789, 0.46015789, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.46015789, 0.        , 0.        ,
        0.        , 0.        , 0.39117625, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.70710678, 0.        , 0.        , 0.        ,
        0.        , 0.70710678, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.51519219, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
      

### LDA 적용

In [37]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=3, random_state=111)   # n_components=3 : 토픽의 개수

# 단어들을 좌표상에 쫙 뿌려놓고 몇 개로 분류를 할 지 정해주면 그 개수만큼 토픽이 생기게 됨.
# 거리가 가까운 단어들끼리 모여서 하나의 토픽에 들어가게 됨.

In [7]:
lda_model.fit(features)   # 교육

LatentDirichletAllocation(n_components=3, random_state=111)

In [31]:
# lda_model.components_ : 토픽의 개수가 3개이므로 배열이 3개가 나옴
topics_list = lda_model.components_[0] # 첫번째 토픽
topics_list   # dictionary_list와 매칭됨. 그래서 이 상태에서 order by를 할 경우 한글과 매칭이 안되므로 두 개를 합친 다음에 order by를 해줘야 한다.

array([0.33440991, 0.79926725, 0.33449536, 0.86083679, 0.33438975,
       0.95573491, 0.33440991, 0.33440991, 0.33452947, 0.33498098,
       0.33452947, 0.33467772, 0.95573491, 1.03622195, 0.79926725,
       0.33438975, 0.79926725, 0.33440991, 0.33438975, 1.03622195,
       0.33438975, 0.33501138, 0.33441824, 0.86083679, 0.33467772,
       0.33498098, 0.33501138, 0.86083679, 0.79926725, 0.33452947,
       1.53902864])

In [30]:
len(lda_model.components_[0])

31

### 토픽별 단어 표시

In [10]:
dictionary_list = vectorizer.get_feature_names_out()
dictionary_list

array(['가다', '가족', '갈다', '건강', '계획', '공부', '나들이', '날씨', '놀다', '마음', '만나다',
       '먹다', '미루다', '배우다', '보내다', '세우다', '시간', '싶다', '어디', '언어', '여행',
       '영화', '오늘', '운동', '음식', '읽다', '정말', '좋아지다', '주말', '친구', '하다'],
      dtype=object)

In [11]:
len(vectorizer.get_feature_names_out())

31

In [12]:
import pandas as pd
df_datas = [topics_list, dictionary_list]
df_topics = pd.DataFrame(data=df_datas)
df_topics

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.33441,0.799267,0.334495,0.860837,0.33439,0.955735,0.33441,0.33441,0.334529,0.334981,...,0.335011,0.334418,0.860837,0.334678,0.334981,0.335011,0.860837,0.799267,0.334529,1.539029
1,가다,가족,갈다,건강,계획,공부,나들이,날씨,놀다,마음,...,영화,오늘,운동,음식,읽다,정말,좋아지다,주말,친구,하다


In [13]:
# 가로를 세로로, 세로를 가로로 만드는 기능
df_topics = df_topics.T
df_topics[:3]

,0,1
0,0.33441,가다
1,0.799267,가족
2,0.334495,갈다


In [14]:
df_topics.columns

RangeIndex(start=0, stop=2, step=1)

In [15]:
# 정렬
df_topics.sort_values(0, ascending=False) # 위에서부터 중요한 키워드순으로 정렬됨

,0,1
30,1.539029,하다
19,1.036222,언어
13,1.036222,배우다
12,0.955735,미루다
5,0.955735,공부
3,0.860837,건강
27,0.860837,좋아지다
23,0.860837,운동
1,0.799267,가족
28,0.799267,주말


### 댓글과 주요 토픽 연결

In [16]:
## 상위 단어 추출
topics_list = list()
for topic in lda_model.components_: # 3개의 토픽이 하나씩 튀어나옴
    df_datas = [topic, dictionary_list] # 2개를 연결시킴
    df_topics = pd.DataFrame(data=df_datas)
    df_topics = df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False) # descending으로 정렬
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:4]) # 상위 4개 추출
    print(topics_text)
    topics_list.append(topics_text)   # 리스트로 만들기

하다 언어 배우다 미루다
오늘 영화 정말 놀다
갈다 읽다 마음 먹다


In [17]:
# get values from series
df_topics[1].values[:4]  # 4는 word의 개수

array(['갈다', '읽다', '마음', '먹다'], dtype=object)

In [18]:
# 시리즈를 하나의 문장으로 생성
' '.join(df_topics[1].values[:4])

'갈다 읽다 마음 먹다'

In [19]:
topics_list_add = [topics_list, ['Topic0','Topic1','Topic2']]
df_topic_keywords = pd.DataFrame(topics_list_add)
df_topic_keywords.T

,0,1
0,하다 언어 배우다 미루다,Topic0
1,오늘 영화 정말 놀다,Topic1
2,갈다 읽다 마음 먹다,Topic2


In [20]:
topics_output = lda_model.transform(features)
topics_output
# 첫번째 댓글은 0.66320011이 가장 큰 값으로 (0,1,2) 중 1에 위치한 토픽으로 분류된다는 것을 알 수 있다.

array([[0.10462299, 0.7905512 , 0.10482582],
       [0.13946382, 0.72088285, 0.13965333],
       [0.12347941, 0.12362429, 0.7528963 ],
       [0.77339137, 0.11328861, 0.11332002],
       [0.75164634, 0.12416304, 0.12419063],
       [0.10458222, 0.10474923, 0.79066855],
       [0.13943843, 0.13959605, 0.72096551],
       [0.11268585, 0.77443415, 0.11288   ],
       [0.72035161, 0.13980797, 0.13984042],
       [0.78957076, 0.10519796, 0.10523128]])

## 각 댓글별 topic 분류

In [21]:
lda_model.n_components

3

In [22]:
df_topics_score = pd.DataFrame(topics_output)  # 각 댓글 당 토픽 분류 점수(가장 높은 값에 해당하는 토픽에 속함)
df_topics_score

,0,1,2
0,0.104623,0.790551,0.104826
1,0.139464,0.720883,0.139653
2,0.123479,0.123624,0.752896
3,0.773391,0.113289,0.113320
4,0.751646,0.124163,0.124191
5,0.104582,0.104749,0.790669
6,0.139438,0.139596,0.720966
7,0.112686,0.774434,0.112880
8,0.720352,0.139808,0.139840
9,0.789571,0.105198,0.105231


In [23]:
# 댓글마다 토픽 분류 지정 (max의 index를 찾음)
import numpy as np
dominate_in_topic = np.argmax(topics_output, axis=1) # 2차원 배열이기 때문에 row 축을 넣어줘야 함
dominate_in_topic

array([1, 1, 2, 0, 0, 2, 2, 1, 0, 0], dtype=int64)

In [27]:
# 해당 댓글이 어느 토픽에 속하는지 알아보기
df_topics_score['dominate_topic'] = dominate_in_topic # 컬럼 추가
df_topics_score[:3]

,0,1,2,dominate_topic,sentences
0,0.104623,0.790551,0.104826,1,오늘 날씨가 좋아서 나들이 가고 싶다.
1,0.139464,0.720883,0.139653,1,이 영화는 정말 재미있었어요.
2,0.123479,0.123624,0.752896,2,맛있는 음식을 먹으러 갈까요?


In [28]:
# 댓글까지 포함해서 알아보기 쉽게 만들기
df_topics_score['sentences'] = sentences
df_topics_score[:3]

,0,1,2,dominate_topic,sentences
0,0.104623,0.790551,0.104826,1,오늘 날씨가 좋아서 나들이 가고 싶다.
1,0.139464,0.720883,0.139653,1,이 영화는 정말 재미있었어요.
2,0.123479,0.123624,0.752896,2,맛있는 음식을 먹으러 갈까요?


In [29]:
df_topics_score

,0,1,2,dominate_topic,sentences
0,0.104623,0.790551,0.104826,1,오늘 날씨가 좋아서 나들이 가고 싶다.
1,0.139464,0.720883,0.139653,1,이 영화는 정말 재미있었어요.
2,0.123479,0.123624,0.752896,2,맛있는 음식을 먹으러 갈까요?
3,0.773391,0.113289,0.113320,0,운동을 하면 건강에 좋아지는 것 같아요.
4,0.751646,0.124163,0.124191,0,공부하기 싫어서 미루고 있어요.
5,0.104582,0.104749,0.790669,2,여행 계획을 세우고 있는데 어디로 갈까요?
6,0.139438,0.139596,0.720966,2,좋은 책을 읽으면 마음이 편안해져요.
7,0.112686,0.774434,0.112880,1,오늘은 친구들과 만나서 재미있게 놀았어요.
8,0.720352,0.139808,0.139840,0,새로운 언어를 배우는 것은 어려워도 흥미로워요.
9,0.789571,0.105198,0.105231,0,주말에 가족들과 함께 시간을 보내기로 했습니다.
